In [13]:
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns


import time
from selenium import webdriver

In [108]:
# Scrape the job listings with Selenium
driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

In [14]:
# pandas display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

In [113]:
# Read in raw email data
filepath = '..\data_local\jobhunt_zapier_3.csv'
df = pd.read_csv(filepath)
df.head()

,email_from,email_subject,email_content,date
0,We Work Remotely,"Listings for Sun, Feb 18",Your daily We Work Remotely job listings\r\n\r...,"Sun, 18 Feb 2018 04:14:19 +0000 (UTC)"
1,We Work Remotely,"Listings for Mon, Feb 19",Your daily We Work Remotely job listings\r\n\r...,"Mon, 19 Feb 2018 04:15:13 +0000 (UTC)"
2,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)"
3,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 19 Feb 2018 17:27:47 -0600 (CST)"
4,LinkedIn Job Alerts,Arina: 503 new jobs for 'data science',LinkedIn | 503 new jobs matching your search f...,"Tue, 20 Feb 2018 01:01:02 +0000 (UTC)"


In [114]:
# testing
df.email_from[0].strip().lower()

'we work remotely'

In [115]:
if df.email_from[0].strip().lower() in ('rodolphe dutel'):
    print('test')

In [69]:
from bs4 import BeautifulSoup

In [137]:
# Returns an array of cleaned links
def parse_email_content(email_from, email_subject, email_content, email_date):
    links_list_clean = []
        
    # different emails have different format
    # process them per email sender
    if email_from.strip().lower() in ('rodolphe dutel'):
        
        # clean up the email content, get the chunk that contains job postings links
        email_content = re.sub(r'\r\n', r'', email_content)
        email_content = re.sub(r'unsubscribe.*', r'', email_content, flags=re.IGNORECASE)
        email_content = re.sub(r'.*-Rodolphe', r'', email_content, flags=re.IGNORECASE)
        
        # extract a list of job post links in the email
        p =r"(?<=---).{1,100} \( ?https://\S* \)-*"
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = re.search(r'(?<=\() ?https?://.*(?=\))', linky, re.IGNORECASE).group().strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//h2[@class="page-title"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="job-company"]/a'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="location"]'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="date-posted"]'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = "//div[contains(concat(' ', @class, ' '), ' job-overview ')]"
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
        
            # get company description
            selector = "//div[contains(concat(' ', @class, ' '), ' job-company-about ')]"
            elems = driver.find_elements_by_xpath(selector)
            company_descr = elems[0].text if len(elems) > 0 else ''
            company_descrs.append(company_descr)
            
    
    elif email_from.strip().lower() in ('indeed job alert', 'indeed'):
        # clean up the email content, get the chunk that contains job postings links
        email_content = re.sub(r'\r\n', r' ', email_content)
        email_content = re.sub(r'Cancel this Recommended Job Alert:.*', r'', email_content, flags=re.IGNORECASE)
        
        # extract a list of job post links in the email
        p =r"https://www.indeed.com/rc/\S* "
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = linky.strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//table[@id="job-content"]//b[@class="jobtitle"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//table[@id="job-content"]//span[@class="company"]'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//table[@id="job-content"]//span[@class="location"]'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//span[@class="date"]'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = '//span[@id="job_summary"]'
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
            
            # get company description
            # go to the company profile page
            click_selector = '//div[@class= "cmp_info"]//div[@class= "cmp_title"]/a'
            elems = driver.find_elements_by_xpath(click_selector)
            if len(elems) > 0:
                elems[0].click()
                # get short company descr
                selector = '//span[@id="cmp-short-description"]'
                elems = driver.find_elements_by_xpath(selector)
                company_descr1 = elems[0].text if len(elems) > 0 else ''
                # get more company descr - with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                soupy = soup.findAll('span',attrs={'id':'cmp-more-description'})
                if soupy:
                    company_descr2 = str(soupy[0])
                    company_descr2 = re.sub(r'<a ?.* id="cmp-less-link">less</a>', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'<span ?.* id="cmp-more-description">', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'</span>', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'(<br>)|(<br/>)', r'\r\n', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'&amp;', r'and', company_descr2, flags=re.IGNORECASE)
                else:
                    company_descr2 = ''
            
                company_descr = company_descr1 + ' ' + company_descr2
            else:
                company_descr = ''
            
            company_descrs.append(company_descr)
            
    
    elif email_from.strip().lower() in ('linkedin job alerts'):
        # clean up the email content, get the chunk that contains job postings links
        # -- LinkedIn job post emails are clean enough as is!
        
        # extract a list of job post links in the email
        p =r"https://www.linkedin.com/comm/jobs/view/\S*"
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = linky.strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//section[@id="top-card"]//h1[@class="title"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//section[@id="top-card"]//span[@class="company"]'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//section[@id="top-card"]//h3[@class="location"]/span/span'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//section[@id="top-card"]//ul[@class="posting-info"]/li[@class="posted"]/span'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = '//section[@id="summary-detail"]//div[@class="description-section"]'
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
            
            # get company description
            selector = '//section[@id="company-lcp"]//div[@class="description-container"]'
            elems = driver.find_elements_by_xpath(selector)
            company_descr = elems[0].text if len(elems) > 0 else ''
            company_descrs.append(company_descr)
            
            
            
            
            
            
            
    # For each job link, record email info
    for linky_url in links_list_clean:
        email_froms.append(email_from)
        email_subjects.append(email_subject)
        email_contents.append(email_content)
        email_dates.append(email_date)

        
    

In [138]:
# testing
job_titles = []
company_names = []
locations = []
date_posteds = []
job_postings = []
company_descrs = []
job_post_links = []

email_froms = []
email_subjects = []
email_contents = []
email_dates = []

# Scrape the job listings with Selenium
driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

# process all emails in the dataframe
for i in range(len(df)):
    parse_email_content(df.email_from[i], df.email_subject[i], df.email_content[i], df.date[i])

# close the Selenium browser
driver.close()

print(job_titles)

['Senior Data Scientist', 'Data Scientist', 'Data Scientist', 'Data Scientist / Signal Researcher', 'Data Scientist Intern', 'Data Scientist - New York', 'Senior Data Scientist', 'Data Scientist Intern', 'Senior Data Scientist', 'Data Scientist', 'Data Scientist - Alexa Language Technologies', 'DATA SCIENTIST', 'Python Developer', 'Data Scientist', 'Analytic Software Engineer', 'Ab Initio Application Lead - Raleigh, NC', 'Big Data Developer', 'Data Warehouse - ETL Developer', 'Director, Data Scientist', 'Staff Data Scientist - Machine Learning', 'Change Management Consultant', 'Data Scientist', 'Enterprise Data Management Consultant', 'Data Scientist', 'Post Doctorate RA - Statistician/Data Scientist', 'Professional-Data Scientist', 'Data Scientist', 'Data Scientist', 'Data Scientist (REMOTE)', 'Data Scientist', 'Data Science', 'Data Scientist', 'Data Science Analytics Leader', 'Data Scientist', 'Director of Data Science - Virtual', 'Data Scientist, Scientific Computing', 'Adjuncts - M

In [131]:
print(company_descrs)

['C2R Ventures is a Talent Advisory Collective dedicated to disrupting the Technology, Financial, FinTech and Management Consulting recruitment space.\n\nOur values and our approach are simple: relationships (both client and candidate) matter above all else.\n\nOur core mission is to represent elite candidates and connect them with businesses who are using Mathematics and Technology as their driving force and competitive advantage.', 'We imagine breakthroughs in the way the world approaches investment management, insurance and related fields by following the scientific method. Our engineers and modelers develop ideas backed by information and improved by iteration. Empowered by extraordinary computing power and vast amounts of data, we build sophisticated predictive models to realize progress.\n\nTwo Sigma is proud to be an equal opportunity workplace. We do not discriminate based upon race, religion, color, national origin, sex, sexual orientation, gender identity/expression, age, sta

In [145]:
df_clean = pd.DataFrame(
    {
     'email_from': email_froms,
     'email_subject': email_subjects,
     'email_content': email_contents,
     'email_date': email_dates,
     'job_title': job_titles,
     'company_name': company_names,
     'location': locations,
     'date_posted': date_posteds,
     'job_posting': job_postings,
     'company_descr': company_descrs,
     'job_post_link': job_post_links
    })
df_clean

,company_descr,company_name,date_posted,email_content,email_date,email_from,email_subject,job_post_link,job_posting,job_title,location
0,At Bluecore we are transforming the way eComme...,Bluecore,3 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=6c7ebde0dbd8...,We are looking for Senior Data Scientists with...,Senior Data Scientist,"New York, NY 10002"
1,,Loadsmart,3 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=59a797491eb2...,"Who we are:\nAt Loadsmart, we won't settle for...",Data Scientist,"New York, NY"
2,,Foursquare,4 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=7490c513ac10...,About Foursquare:\nSince our inception in 2009...,Data Scientist,"New York, NY"
3,About HRG\n\nHRG's TRADING TEAM recruits the t...,"The Hagan-Ricci Group, Inc.",5 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=4eceac18fce0...,A small and very successful quantitative fund ...,Data Scientist / Signal Researcher,"New York, NY"
4,,Oath Inc,5 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=6b82f29c6741...,"Oath, a subsidiary of Verizon, is a values-led...",Data Scientist Intern,"New York, NY"
5,,BenevolentAI,9 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=89a671ea11d5...,Data Scientist – New York\n\nSummary\nThe Data...,Data Scientist - New York,"New York, NY"
6,We’re proud to be an industry leader in health...,Weight Watchers International,5 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=1c9df0f23736...,"At Weight Watchers, we inspire and guide the h...",Senior Data Scientist,"New York, NY"
7,,SphereOI Studios,6 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=21cec82830aa...,Job Summary\nSphereOI is seeking a Data Scient...,Data Scientist Intern,"McLean, VA"
8,,Capgemini: RO,3 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=0c3d5016781c...,Short Description\n\nAbout Capgemini\n\nWith m...,Senior Data Scientist,"New York, NY"
9,Twitter is an online news and social networkin...,Twitter,11 days ago,Recommended Jobs for You Jobs 1 to 23 of 23 r...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,https://www.indeed.com/viewjob?jk=4a3134853e77...,A rare opportunity to analyze petabytes of dat...,Data Scientist,"San Francisco, CA 94103"


In [146]:
# Pickle the cleaned job posts dataset
df_clean.to_pickle('../data_local/job_posts_clean.pkl')

In [147]:
len(df_clean)

183

In [148]:
len(df)

14

In [149]:
df

,email_from,email_subject,email_content,date
0,We Work Remotely,"Listings for Sun, Feb 18",Your daily We Work Remotely job listings\r\n\r...,"Sun, 18 Feb 2018 04:14:19 +0000 (UTC)"
1,We Work Remotely,"Listings for Mon, Feb 19",Your daily We Work Remotely job listings\r\n\r...,"Mon, 19 Feb 2018 04:15:13 +0000 (UTC)"
2,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)"
3,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 19 Feb 2018 17:27:47 -0600 (CST)"
4,LinkedIn Job Alerts,Arina: 503 new jobs for 'data science',LinkedIn | 503 new jobs matching your search f...,"Tue, 20 Feb 2018 01:01:02 +0000 (UTC)"
5,We Work Remotely,"Listings for Tue, Feb 20",Your daily We Work Remotely job listings\r\n\r...,"Tue, 20 Feb 2018 04:21:59 +0000 (UTC)"
6,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 9 of...,"Tue, 20 Feb 2018 13:51:08 -0600 (CST)"
7,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Tue, 20 Feb 2018 19:23:41 -0600 (CST)"
8,LinkedIn Job Alerts,Arina: 776 new jobs for 'data science',LinkedIn | 776 new jobs matching your search f...,"Wed, 21 Feb 2018 02:12:09 +0000 (UTC)"
9,We Work Remotely,"Listings for Wed, Feb 21",Your daily We Work Remotely job listings\r\n\r...,"Wed, 21 Feb 2018 04:44:32 +0000 (UTC)"


In [144]:
df_clean.email_from.value_counts()

Indeed                 96
Indeed Job Alert       54
LinkedIn Job Alerts    33
Name: email_from, dtype: int64